In [27]:
import pandas as pd
import numpy as np
import warnings
import time
import random
from tqdm import tqdm
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
def req(url):
    import requests
    from bs4 import BeautifulSoup as bs4
    response = requests.get(url)
    contents = response.content
    soup = bs4(contents, "html.parser")
    return soup

In [3]:
def fill_area(row):
    if row["adresse"] == "":
        return row["area"]
    else:
        return row["adresse"]

In [4]:
def rent_bin(row):
    if row["rent"] < 600:
        return "below 600"
    elif (row["rent"] >= 600) & (row["rent"] < 1000):
        return "600 to 1000"
    elif (row["rent"] >= 1000) & (row["rent"] < 1500):
        return "1000 to 1500"
    elif (row["rent"] >= 1500) & (row["rent"] < 2000):
        return "1500 to 2000"
    elif (row["rent"] >= 2000) & (row["rent"] < 3000):
        return "2000 to 3000"
    else:
        return "over 3000"

In [20]:
df = pd.read_csv("data raw new.csv")
df["adresse"] = df["adresse"].str.replace("berlin","Berlin")

In [21]:
df.head()

,link,adresse,rent,rooms,size
0,https://www.immowelt.de/expose/2aqr554,Berlin (Kreuzberg),620 €,3 Zi.,83 m²
1,https://www.immowelt.de/expose/2abw253,Berlin (Friedrichshain),503 €,2.5 Zi.,55 m²
2,https://www.immowelt.de/expose/2a79752,"Stralauer Allee 14a, Berlin (Friedrichshain)",1.750 €,2 Zi.,59 m²
3,https://www.immowelt.de/expose/2akzn54,Berlin (Charlottenburg),1.340 €,2 Zi.,85.07 m²
4,https://www.immowelt.de/expose/29yzr5s,Berlin (Neukölln),3.258 €,4 Zi.,181.95 m²


In [22]:
df.isna().sum()

link       0
adresse    0
rent       0
rooms      4
size       3
dtype: int64

In [23]:
df = df.dropna(axis = 0)
df.isna().sum()

link       0
adresse    0
rent       0
rooms      0
size       0
dtype: int64

In [24]:
adresse,area = [],[]
for i in df["adresse"]:
    adresse.append(i.split("Berlin ")[0].strip(" "))
    try:
        area.append(i.split("(")[1].strip(" "))
    except:
        area.append("")
size = []
for i in df["size"]:
    size.append(i.split(" m")[0])

room = []
for i in df["rooms"]:
    room.append(i.split(" Z")[0])


In [25]:
df["rooms"] = room
df["rooms"] = df["rooms"].astype(float)
df["size"] = size
df["size"] = df["size"].astype(float)
df["adresse"] = adresse
df["area"] = area
df["adresse"] = df["adresse"].str.replace(",","")
df["rent"] = df["rent"].str.replace("€","")
df["rent"] = df["rent"].str.replace(".","")
df["rent"] = df["rent"].str.replace(",",".")

In [33]:
rent = []
for i in df["rent"]:
    if i =="auf Anfrage":
        rent.append(np.nan)
    else:
        rent.append(i)
df["rent"] = rent
df = df.dropna(axis = 0)
df.isna().sum()

link       0
adresse    0
rent       0
rooms      0
size       0
area       0
dtype: int64

In [34]:
df["rent"] = df["rent"].astype(float)
df["area"] = df["area"].str.replace("(","").str.replace(")","").str.replace("/ ","")
df["adresse"] = df.apply(fill_area, axis = 1)
df["rent_range"] = df.apply(rent_bin, axis = 1)


In [35]:
df.head()

,link,adresse,rent,rooms,size,area,rent_range
0,https://www.immowelt.de/expose/2aqr554,Kreuzberg,620.0,3.0,83.00,Kreuzberg,600 to 1000
1,https://www.immowelt.de/expose/2abw253,Friedrichshain,503.0,2.5,55.00,Friedrichshain,below 600
2,https://www.immowelt.de/expose/2a79752,Stralauer Allee 14a,1750.0,2.0,59.00,Friedrichshain,1500 to 2000
3,https://www.immowelt.de/expose/2akzn54,Charlottenburg,1340.0,2.0,85.07,Charlottenburg,1000 to 1500
4,https://www.immowelt.de/expose/29yzr5s,Neukölln,3258.0,4.0,181.95,Neukölln,over 3000


In [36]:
df["link"][df["rent"]==df["rent"].max()].iloc[0]

'https://www.immowelt.de/expose/29rvk5s'

In [37]:
df["link"].iloc[0]

'https://www.immowelt.de/expose/2aqr554'

In [38]:
df.shape[0]

661

In [39]:
df.to_csv("data clean new.csv", index = False)

## DONT RUN THIS CELL - scraping part

In [40]:
# df0 = pd.DataFrame()
# for j in tqdm(range(df.shape[0])):
#     time.sleep(random.randint(4,7))
#     soup = req(df["link"].iloc[j])
#     infobox = soup.find("app-estate-object-informations")
#     d = {}
#     try:
#         infos = infobox.find_all("li",{"class":"ng-star-inserted"})
#         for i in infos:
#             i = i.text
#             if ":" in i:
#                 i1 = i.split(":")[0]
#                 i2 = i.split(":")[1].strip()
#                 d[i1] = [i2]
#             else:
#                 d[i] = [1]
#     except:
#         d["not available"] = [1]
#     df1 = pd.DataFrame.from_dict(d)
#     df0 = pd.concat([df0,df1])
# df0.shape

100%|██████████| 661/661 [1:04:41<00:00,  5.87s/it]


(659, 87)

In [42]:
display(df0.head(2))
display(df0.shape)

,Bad mit Wanne,Böden,Weitere Räume,Haustiere erlaubt,"Balkon, Terrasse",Einbauküche,Bad mit Dusche,"Balkon, Garten",barrierefrei,Zustand,...,"barrierefrei, WG-geeignet","Pantry, Speisekammer",Stellplätze,3 Stellplätze,"Garten, Wintergarten","Einbauküche, Pantry, Speisekammer","Garten, Gartenmitbenutzung","Balkon, Wintergarten","Loggia, Terrasse",teilweise klimatisiert
0,1.0,"Holzdielen, Fliesenboden",Kelleranteil,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,Laminat,Kelleranteil,1.0,1.0,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


(659, 87)

In [44]:
df0 = df0.reset_index(drop = True)
df_raw = pd.concat([df,df0], axis = 1)
try:
    df_raw = df_raw.drop(columns = ["level_0"])
except:
    pass
df_raw.head()

,link,adresse,rent,rooms,size,area,rent_range,Bad mit Wanne,Böden,Weitere Räume,...,"barrierefrei, WG-geeignet","Pantry, Speisekammer",Stellplätze,3 Stellplätze,"Garten, Wintergarten","Einbauküche, Pantry, Speisekammer","Garten, Gartenmitbenutzung","Balkon, Wintergarten","Loggia, Terrasse",teilweise klimatisiert
0,https://www.immowelt.de/expose/2aqr554,Kreuzberg,620.0,3.0,83.00,Kreuzberg,600 to 1000,1.0,"Holzdielen, Fliesenboden",Kelleranteil,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.immowelt.de/expose/2abw253,Friedrichshain,503.0,2.5,55.00,Friedrichshain,below 600,NaN,Laminat,Kelleranteil,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://www.immowelt.de/expose/2a79752,Stralauer Allee 14a,1750.0,2.0,59.00,Friedrichshain,1500 to 2000,NaN,Parkett,Kelleranteil,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://www.immowelt.de/expose/2akzn54,Charlottenburg,1340.0,2.0,85.07,Charlottenburg,1000 to 1500,1.0,"Fliesenboden, Parkett",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://www.immowelt.de/expose/29yzr5s,Neukölln,3258.0,4.0,181.95,Neukölln,over 3000,NaN,NaN,Kelleranteil,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
df_raw.to_csv("data full raw new.csv", index = False)

## START FROM HERE

In [180]:
df_raw = pd.read_csv("data full raw new.csv")

In [181]:
def encode(df, cols):
    import numpy as np
    el0 = []
    for i in range(df.shape[0]):
        el0.append(0)
    el0 = np.array(el0)
    for col in cols:
        el1 = []
        if col in df.columns.tolist():
            for i in df[col]:
                if i == 1.0:
                    el1.append(1)
                else:
                    el1.append(0)
        else:
            el1.append(0)
        el1 = np.array(el1)
        el0 = el0+el1

    el = []
    for i in el0:
        if i > 0:
            el.append(1)
        else:
            el.append(0)
    return el

### cleaning (1)

In [182]:
lguest_wc = []
col_guest_wc = ["Bad mit Wanne, Gäste-WC ","Bad mit Wanne und Dusche, Gäste-WC ",'Gäste-WC ',
    'Bad mit Fenster und Wanne, Gäste-WC ','Bad mit Dusche, Gäste-WC ','Bad mit Fenster, Wanne und Dusche, Gäste-WC ',
    'Bad mit Fenster, Gäste-WC ','Bad mit Fenster und Dusche, Gäste-WC ',
    'Bad mit Fenster, Wanne und Dusche, Gäste-WC, Bad/WC getrennt ',
    'Bad mit Fenster, Wanne und Dusche, Gäste-WC, Bad/WC getrennt, Bidet ']
lguest_wc = encode(df_raw, col_guest_wc)

lshower = []
col_shower = ["Bad mit Wanne und Dusche ",'Bad mit Dusche ','Bad mit Wanne und Dusche, Gäste-WC ',
    'Bad mit Dusche, Gäste-WC ','Bad mit Fenster, Wanne und Dusche, Gäste-WC ','Bad mit Fenster, Wanne und Dusche ',
    'Bad mit Fenster, Gäste-WC ','Bad mit Fenster, Wanne und Dusche, Gäste-WC, Bad/WC getrennt ',
    'Bad mit Fenster und Dusche, Gäste-WC ','Bad mit Fenster, Wanne und Dusche, Gäste-WC, Bad/WC getrennt, Bidet ',
    'Bad mit Fenster, Wanne und Dusche, Bad/WC getrennt, Bidet ']
lshower = encode(df_raw, col_shower)

lbathtub = []
col_bathtub = ["Bad mit Wanne und Dusche ",'Bad mit Wanne, Gäste-WC ','Bad mit Fenster und Wanne ',
    'Bad mit Wanne und Dusche, Gäste-WC ','Bad mit Fenster und Wanne, Gäste-WC ','Bad mit Wanne ',
    'Bad mit Fenster, Wanne und Dusche, Gäste-WC ','Bad mit Fenster, Wanne und Dusche ',
    'Bad mit Fenster, Wanne und Dusche, Gäste-WC, Bad/WC getrennt ',
    'Bad mit Fenster, Wanne und Dusche, Gäste-WC, Bad/WC getrennt, Bidet ',
    'Bad mit Fenster, Wanne und Dusche, Bad/WC getrennt, Bidet ']
lbathtub = encode(df_raw, col_bathtub)

lbathroom_window = []
col_bathroom_window = ['Bad mit Fenster ','Bad mit Fenster und Wanne ','Bad mit Fenster und Wanne, Gäste-WC ',
    'Bad mit Fenster, Wanne und Dusche, Gäste-WC ','Bad mit Fenster und Dusche ','Bad mit Fenster, Wanne und Dusche ',
    'Bad mit Fenster, Gäste-WC ','Bad mit Fenster, Wanne und Dusche, Gäste-WC, Bad/WC getrennt ',
    'Bad mit Fenster und Dusche, Gäste-WC ','Bad mit Fenster, Wanne und Dusche, Gäste-WC, Bad/WC getrennt, Bidet ',
    'Bad mit Fenster, Wanne und Dusche, Bad/WC getrennt, Bidet ']
lbathroom_window = encode(df_raw, col_bathroom_window)

lbarrier_free = []
col_barrier_free = ['barrierefrei, rollstuhlgerecht ','barrierefrei ','barrierefrei, rollstuhlgerecht, WG-geeignet ',
    'barrierefrei, WG-geeignet ','barrierefrei, als Ferienimmobilie geeignet ']
lbarrier_free = encode(df_raw, col_barrier_free)

lwheelchair_friendly = []
col_wheelchair_friendly = ['barrierefrei, rollstuhlgerecht ','rollstuhlgerecht ',
    'barrierefrei, rollstuhlgerecht, WG-geeignet ']
lwheelchair_friendly = encode(df_raw, col_wheelchair_friendly)

lapartment_sharing = []
col_apartment_sharing = ['barrierefrei, rollstuhlgerecht, WG-geeignet ','barrierefrei, WG-geeignet ','WG-geeignet ']
lapartment_sharing = encode(df_raw, col_apartment_sharing)

lbalcony = []
col_balcony = ['Balkon ','Balkon, Garten, Gartenmitbenutzung ','Balkon, Terrasse ','Balkon, Terrasse ',
    'Balkon, Garten ','Balkon, Terrasse, Gartenmitbenutzung ','Balkon, Gartenmitbenutzung ',
    'Balkon, Garten, Terrasse ','Balkon, Garten, Terrasse, Gartenanteil ','Balkon, Wintergarten ',
    'Balkon, Garten, Terrasse, Gartenmitbenutzung ','Balkon, Garten, Terrasse, Gartenanteil, Gartenmitbenutzung ',
    'Balkon, Terrasse, Gartenanteil ','Balkon, Garten, Loggia, Terrasse ','Balkon, Loggia, Terrasse ']
lbalcony = encode(df_raw, col_balcony)

lgarden = []
col_garden = ['Garten ','Balkon, Garten, Gartenmitbenutzung ','Gartenmitbenutzung ','Terrasse, Gartenmitbenutzung ',
    'Balkon, Garten ','Garten, Terrasse ','Balkon, Terrasse, Gartenmitbenutzung ','Garten, Wintergarten '
    'Balkon, Gartenmitbenutzung ','Garten, Terrasse, Gartenmitbenutzung ','Balkon, Garten, Terrasse ','garden',
    'Balkon, Garten, Terrasse, Gartenanteil ','Garten, Terrasse, Gartenanteil ','Garten, Gartenmitbenutzung ',
    'Balkon, Garten, Terrasse, Gartenmitbenutzung ','Balkon, Garten, Terrasse, Gartenanteil, Gartenmitbenutzung ',
    'Balkon, Garten, Loggia, Terrasse ','Terrasse, Wintergarten ']
lgarden = encode(df_raw, col_garden)

lgarden_sharing = []
col_garden_sharing = ['Balkon, Garten, Gartenmitbenutzung ','Gartenmitbenutzung ','Terrasse, Gartenmitbenutzung ',
    'Balkon, Terrasse, Gartenmitbenutzung ','Terrasse, Gartenanteil ','Balkon, Gartenmitbenutzung ',
    'Garten, Terrasse, Gartenmitbenutzung ','Balkon, Garten, Terrasse, Gartenanteil ',
    'Garten, Terrasse, Gartenanteil ','Garten, Gartenmitbenutzung ','Balkon, Garten, Terrasse, Gartenmitbenutzung ',
    'Balkon, Garten, Terrasse, Gartenanteil, Gartenmitbenutzung ','Balkon, Terrasse, Gartenanteil ']
lgarden_sharing = encode(df_raw, col_garden_sharing)

lterrace = []
col_terrace = ['Balkon, Terrasse ','Terrasse, Gartenmitbenutzung ','Garten, Terrasse ','Terrasse ',
    'Balkon, Terrasse, Gartenmitbenutzung ','Terrasse, Gartenanteil ','Garten, Terrasse, Gartenmitbenutzung ',
    'Balkon, Garten, Terrasse ','Balkon, Garten, Terrasse, Gartenanteil ','Garten, Terrasse, Gartenanteil ',
    'Balkon, Garten, Terrasse, Gartenmitbenutzung ','Balkon, Garten, Terrasse, Gartenanteil, Gartenmitbenutzung ',
    'Balkon, Terrasse, Gartenanteil ','Balkon, Garten, Loggia, Terrasse ','Balkon, Loggia, Terrasse ',
    'Loggia, Terrasse ','Terrasse, Wintergarten ']
lterrace = encode(df_raw, col_terrace)

lloggia = []
col_loggia = ['Balkon, Garten, Loggia, Terrasse ','Balkon, Loggia, Terrasse ','Loggia, Terrasse ','Loggia ']
lloggia = encode(df_raw, col_loggia)

lfitted_kitchen = []
col_fitted_kitchen = ['Einbauküche ','Einbauküche, offene Küche ','Einbauküche, offene Küche, Pantry, Speisekammer ',
    'Einbauküche, offene Küche, Speisekammer ','Einbauküche, Pantry, Speisekammer ','Einbauküche, Speisekammer ']
lfitted_kitchen = encode(df_raw, col_fitted_kitchen)

lopen_kitchen = []
col_open_kitchen = ['Einbauküche, offene Küche ','offene Küche ','Einbauküche, offene Küche, Pantry, Speisekammer ',
    'Einbauküche, offene Küche, Speisekammer ']
lopen_kitchen = encode(df_raw, col_open_kitchen)

lpantry = []
col_pantry = ['Pantry ','Pantry, Speisekammer ','Einbauküche, offene Küche, Pantry, Speisekammer ',
    'Einbauküche, offene Küche, Pantry, Speisekammer ','Einbauküche, Pantry, Speisekammer ',
    'Einbauküche, Speisekammer ','Speisekammer ']
lpantry = encode(df_raw, col_pantry)

lparkspace = []
col_parkspace = ['1 Stellplatz ','1 Stellplatz','Stellplatz','Stellplatz ']
lparkspace = encode(df_raw, col_parkspace)

lmore_parkspaces = []
col_more_parkspaces = ['Stellplätze','2 Stellplätze ','2 Stellplätze','3 Stellplätze','4 Stellplätze ']
lmore_parkspaces = encode(df_raw, col_more_parkspaces)

lperson_elevator = []
col_person_elevator = ['Personenaufzug ','Personenaufzug, Lastenaufzug ']
lperson_elevator = encode(df_raw, col_person_elevator)

slservice_elevator = []
col_service_elevator = ['Lastenaufzug ','Personenaufzug, Lastenaufzug ']
lservice_elevator = encode(df_raw, col_service_elevator)

In [183]:
df_new = df_raw.copy()
df_new["guest_wc"] = lguest_wc
df_new["shower"] = lshower
df_new["bathtub"] = lbathtub
df_new["bathroom_window"] = lbathroom_window
df_new["barrier_free"] = lbarrier_free
df_new["wheelchair_friendly"] = lwheelchair_friendly
df_new["apartment_sharing"] = lapartment_sharing
df_new["balcony"] = lbalcony
df_new["garden"] = lgarden
df_new["garden_sharing"] = lgarden_sharing
df_new["terrace"] = lterrace
df_new["fitted_kitchen"] = lfitted_kitchen
df_new["open_kitchen"] = lopen_kitchen
df_new["pantry"] = lpantry
df_new["parkspace"] = lparkspace
df_new["more_parkspaces"] = lmore_parkspaces
df_new["person_elevator"] = lperson_elevator
df_new["service_elevator"] = lservice_elevator


In [184]:
columns_to_drop = ['Bad mit Fenster ','Bad mit Wanne und Dusche ','Bad mit Dusche ','Bad mit Wanne, Gäste-WC ',
    'Bad mit Fenster und Wanne ','Bad mit Wanne und Dusche, Gäste-WC ','Bad mit Fenster und Wanne, Gäste-WC ',
    'Bad mit Wanne ','Bad mit Dusche, Gäste-WC ','Bad mit Fenster, Wanne und Dusche, Gäste-WC ','Stellplatz ',
    'Gäste-WC ','Bad mit Fenster und Dusche ','Bad mit Fenster, Wanne und Dusche ','Bad mit Fenster, Gäste-WC ',
    'Bad mit Fenster, Wanne und Dusche, Gäste-WC, Bad/WC getrennt ','Bad mit Fenster und Dusche, Gäste-WC ',
    'barrierefrei, rollstuhlgerecht ','barrierefrei, rollstuhlgerecht, WG-geeignet ','Garten, Wintergarten ',
    'rollstuhlgerecht ','barrierefrei, WG-geeignet ','barrierefrei ','WG-geeignet ','Loggia, Terrasse ',
    'Balkon ','Garten ','Balkon, Garten, Gartenmitbenutzung ','Gartenmitbenutzung ','Einbauküche, Speisekammer ',
    'Balkon, Terrasse ','Terrasse, Gartenmitbenutzung ','Balkon, Garten ','Terrasse ','4 Stellplätze ',
    'Garten, Terrasse ','Balkon, Terrasse, Gartenmitbenutzung ','Terrasse, Gartenanteil ','2 Stellplätze',
    'Balkon, Gartenmitbenutzung ','Garten, Terrasse, Gartenmitbenutzung ','Balkon, Garten, Loggia, Terrasse ',
    'Balkon, Garten, Terrasse ','Balkon, Garten, Terrasse, Gartenanteil ','Balkon, Loggia, Terrasse ','Loggia ',
    'Garten, Terrasse, Gartenanteil ','Garten, Gartenmitbenutzung ','Balkon, Wintergarten ',
    'Einbauküche ','Einbauküche, offene Küche ','offene Küche ','Pantry ','Einbauküche, Pantry, Speisekammer ',
    'Pantry, Speisekammer ','Einbauküche, offene Küche, Pantry, Speisekammer ','Personenaufzug ',
    'Einbauküche, offene Küche, Speisekammer ','1 Stellplatz ','1 Stellplatz','Terrasse, Wintergarten ',
    'Stellplatz','Stellplätze', '2 Stellplätze ','Balkon, Garten, Terrasse, Gartenmitbenutzung ','Speisekammer ',
    'Balkon, Garten, Terrasse, Gartenanteil, Gartenmitbenutzung ','Balkon, Terrasse, Gartenanteil ',
    'Bad mit Fenster, Wanne und Dusche, Gäste-WC, Bad/WC getrennt, Bidet ','3 Stellplätze',
    'Bad mit Fenster, Wanne und Dusche, Bad/WC getrennt, Bidet ','barrierefrei, als Ferienimmobilie geeignet ',
    'Personenaufzug, Lastenaufzug ','Lastenaufzug ','kontrollierte Be- und Entlüftungsanlage ']
print(df_new.shape[1])
for i in columns_to_drop:
    try:
        df_new = df_new.drop(columns = [i])
    except:
        pass
print(df_new.shape[1])

112
41


In [185]:
c = 0
for i in df_raw.columns:
    if "Speisekammer" in i:
        c+=1
c

6

In [186]:
df_new.columns

Index(['link', 'adresse', 'rent', 'rooms', 'size', 'area', 'rent_range',
       'Böden', 'Weitere Räume', 'Haustiere erlaubt ', 'Zustand', 'Baujahr',
       'Ausstattung', 'voll klimatisiert ', 'Fenster', 'Sicherheitstechnik',
       'Anschlüsse', 'Ausblick', 'Serviceleistungen', 'not available',
       'als Ferienimmobilie geeignet ', 'Wellness', 'teilweise klimatisiert ',
       'guest_wc', 'shower', 'bathtub', 'bathroom_window', 'barrier_free',
       'wheelchair_friendly', 'apartment_sharing', 'balcony', 'garden',
       'garden_sharing', 'terrace', 'fitted_kitchen', 'open_kitchen', 'pantry',
       'parkspace', 'more_parkspaces', 'person_elevator', 'service_elevator'],
      dtype='object')

In [187]:
df_new = df_new.rename(columns={'Haustiere erlaubt ': 'pets_allowed', 'Personenaufzug ': 'person_elevator',
                        'Baujahr':'year_constructed','Weitere Räume':'other_rooms','Fenster':'window',
                        'Lastenaufzug ':'service_elevator','Serviceleistungen':'services',
                        'Sicherheitstechnik':'security','Zustand':'condition','Ausstattung':'furnishing',
                        'teilweise klimatisiert ':'partly_air_conditioned','Loggia ':'loggia',
                        'voll klimatisiert ':'fully_air_conditioned','Böden':'floors','Anschlüsse':'connection',
                        'Ausblick':'view','adresse':'addresse','Wellness':'wellness','not available':'not_available',
                        'als Ferienimmobilie geeignet ':'vacation_property'})

In [188]:
df_new.columns

Index(['link', 'addresse', 'rent', 'rooms', 'size', 'area', 'rent_range',
       'floors', 'other_rooms', 'pets_allowed', 'condition',
       'year_constructed', 'furnishing', 'fully_air_conditioned', 'window',
       'security', 'connection', 'view', 'services', 'not_available',
       'vacation_property', 'wellness', 'partly_air_conditioned', 'guest_wc',
       'shower', 'bathtub', 'bathroom_window', 'barrier_free',
       'wheelchair_friendly', 'apartment_sharing', 'balcony', 'garden',
       'garden_sharing', 'terrace', 'fitted_kitchen', 'open_kitchen', 'pantry',
       'parkspace', 'more_parkspaces', 'person_elevator', 'service_elevator'],
      dtype='object')

In [189]:
pd.set_option('display.max_columns', None)
df_new.head()

,link,addresse,rent,rooms,size,area,rent_range,floors,other_rooms,pets_allowed,condition,year_constructed,furnishing,fully_air_conditioned,window,security,connection,view,services,not_available,vacation_property,wellness,partly_air_conditioned,guest_wc,shower,bathtub,bathroom_window,barrier_free,wheelchair_friendly,apartment_sharing,balcony,garden,garden_sharing,terrace,fitted_kitchen,open_kitchen,pantry,parkspace,more_parkspaces,person_elevator,service_elevator
0,https://www.immowelt.de/expose/2aqr554,Kreuzberg,620.0,3.0,83.00,Kreuzberg,600 to 1000,"Holzdielen, Fliesenboden",Kelleranteil,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,https://www.immowelt.de/expose/2abw253,Friedrichshain,503.0,2.5,55.00,Friedrichshain,below 600,Laminat,Kelleranteil,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0
2,https://www.immowelt.de/expose/2a79752,Stralauer Allee 14a,1750.0,2.0,59.00,Friedrichshain,1500 to 2000,Parkett,Kelleranteil,NaN,renoviert / saniert,2019.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,0,0,1,0,0,1,1,0,0,1,0,0,0,0,0,0
3,https://www.immowelt.de/expose/2akzn54,Charlottenburg,1340.0,2.0,85.07,Charlottenburg,1000 to 1500,"Fliesenboden, Parkett",NaN,NaN,NaN,1880.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0
4,https://www.immowelt.de/expose/29yzr5s,Neukölln,3258.0,4.0,181.95,Neukölln,over 3000,NaN,Kelleranteil,NaN,Neubau,2021.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,1,0,0,0,1,1,0,0,1,1,0,0,0,0,1,0


In [190]:
df_new["floors"].unique()

array(['Holzdielen, Fliesenboden', 'Laminat', 'Parkett',
       'Fliesenboden, Parkett', nan, 'Fliesenboden', 'Holzdielen',
       'Fliesenboden, Kunststoffboden', 'Fliesenboden, Laminat',
       'Kunststoffboden, Parkett', 'Holzdielen, Fliesenboden, Parkett',
       'Teppichboden', 'Fliesenboden, Linoleum', 'Holzdielen, Parkett',
       'Kunststoffboden, Laminat', 'Estrich',
       'Fliesenboden, Laminat, Parkett', 'Holzdielen, Steinboden',
       'Fliesenboden, Teppichboden',
       'Holzdielen, Fliesenboden, Parkett, Teppichboden',
       'Kunststoffboden', 'Fliesenboden, Parkett, Steinboden',
       'Parkett, Steinboden', 'Marmorboden, Parkett',
       'Holzdielen, Marmorboden'], dtype=object)

In [191]:
df_new["floors"] = df_new["floors"].fillna("no information")

In [192]:
df_new["floors"].unique()

array(['Holzdielen, Fliesenboden', 'Laminat', 'Parkett',
       'Fliesenboden, Parkett', 'no information', 'Fliesenboden',
       'Holzdielen', 'Fliesenboden, Kunststoffboden',
       'Fliesenboden, Laminat', 'Kunststoffboden, Parkett',
       'Holzdielen, Fliesenboden, Parkett', 'Teppichboden',
       'Fliesenboden, Linoleum', 'Holzdielen, Parkett',
       'Kunststoffboden, Laminat', 'Estrich',
       'Fliesenboden, Laminat, Parkett', 'Holzdielen, Steinboden',
       'Fliesenboden, Teppichboden',
       'Holzdielen, Fliesenboden, Parkett, Teppichboden',
       'Kunststoffboden', 'Fliesenboden, Parkett, Steinboden',
       'Parkett, Steinboden', 'Marmorboden, Parkett',
       'Holzdielen, Marmorboden'], dtype=object)

In [193]:
def encode_floors(df):
    import numpy as np
    parquet,carpet,tiles,laminate,screed,wooden_floor,synthetic,linoleum,marble = [],[],[],[],[],[],[],[],[]
    for i in df['floors']:
        if "Parkett" in i:
            parquet.append(1)
        else:
            parquet.append(0)
        if "Teppichboden" in i:
            carpet.append(1)
        else:
            carpet.append(0)
        if "Fliesenboden" in i:
            tiles.append(1)
        else:
            tiles.append(0)
        if "Laminat" in i:
            laminate.append(1)
        else:
            laminate.append(0)
        if "Estrich" in i:
            screed.append(1)
        else:
            screed.append(0)
        if "Holzdielen" in i:
            wooden_floor.append(1)
        else:
            wooden_floor.append(0)
        if "Kunststoffboden" in i:
            synthetic.append(1)
        else:
            synthetic.append(0)
        if "Linoleum" in i:
            linoleum.append(1)
        else:
            linoleum.append(0)
        if "Marmorboden" in i:
            marble.append(1)
        else:
            marble.append(0)    
    # parquet,carpet,tiles,laminate,screed,wooden_floor,synthetic,linoleum,marble
    df["parquet"] = parquet
    df["carpet"] = carpet
    df["tiles"] = tiles
    df["laminate"] = laminate
    df["screed"] = screed
    df["wooden_floor"] = wooden_floor
    df["synthetic"] = synthetic
    df["linoleum"] = linoleum
    df["marble"] = marble
    df = df.drop(columns = ["floors"])
    return df

In [194]:
df_new = encode_floors(df_new)

In [195]:
df_new.columns

Index(['link', 'addresse', 'rent', 'rooms', 'size', 'area', 'rent_range',
       'other_rooms', 'pets_allowed', 'condition', 'year_constructed',
       'furnishing', 'fully_air_conditioned', 'window', 'security',
       'connection', 'view', 'services', 'not_available', 'vacation_property',
       'wellness', 'partly_air_conditioned', 'guest_wc', 'shower', 'bathtub',
       'bathroom_window', 'barrier_free', 'wheelchair_friendly',
       'apartment_sharing', 'balcony', 'garden', 'garden_sharing', 'terrace',
       'fitted_kitchen', 'open_kitchen', 'pantry', 'parkspace',
       'more_parkspaces', 'person_elevator', 'service_elevator', 'parquet',
       'carpet', 'tiles', 'laminate', 'screed', 'wooden_floor', 'synthetic',
       'linoleum', 'marble'],
      dtype='object')

In [196]:
cols = ["not_available","pets_allowed","person_elevator","other_rooms","window","connection",
        "partly_air_conditioned","view","pantry","service_elevator","security","fully_air_conditioned",
        "wellness","services"]
for i in cols:
    df_new[i].fillna(0, inplace = True)

In [197]:
cols = ["furnishing","condition"]
for i in cols:
    df_new[i].fillna("no information", inplace = True)

In [198]:
furnishing = []
for i in df_new["furnishing"]:
    i = i.replace("möbliert","furnished").replace("neuwertig","like new").replace("teilweise","partly")
    furnishing.append(i)
df_new["furnishing"] = furnishing

In [199]:
condition = []
for i in df_new["condition"]:
    i = i.replace('renoviert / saniert',"renovated").replace("Neubau","new building").replace("Erstbezug","first-time-use").replace("Altbau (bis 1945)","old building").replace("gepflegt","well-maintained")
    condition.append(i)
df_new["condition"] = condition

In [200]:
print(df_new.shape[0])
df_new = df_new[df_new["not_available"] != 1.0]
print(df_new.shape[0])

667
660


In [201]:
cols = ["index","not_available","vacation_property"]
for i in cols:
    try:
        df_new = df_new.drop(columns = [i])
    except:
        pass

In [202]:
df_new.head()

,link,addresse,rent,rooms,size,area,rent_range,other_rooms,pets_allowed,condition,year_constructed,furnishing,fully_air_conditioned,window,security,connection,view,services,wellness,partly_air_conditioned,guest_wc,shower,bathtub,bathroom_window,barrier_free,wheelchair_friendly,apartment_sharing,balcony,garden,garden_sharing,terrace,fitted_kitchen,open_kitchen,pantry,parkspace,more_parkspaces,person_elevator,service_elevator,parquet,carpet,tiles,laminate,screed,wooden_floor,synthetic,linoleum,marble
0,https://www.immowelt.de/expose/2aqr554,Kreuzberg,620.0,3.0,83.00,Kreuzberg,600 to 1000,Kelleranteil,1.0,no information,NaN,no information,0.0,0,0,0,0,0,0,0.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0
1,https://www.immowelt.de/expose/2abw253,Friedrichshain,503.0,2.5,55.00,Friedrichshain,below 600,Kelleranteil,1.0,no information,NaN,no information,0.0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,https://www.immowelt.de/expose/2a79752,Stralauer Allee 14a,1750.0,2.0,59.00,Friedrichshain,1500 to 2000,Kelleranteil,0.0,renovated,2019.0,no information,0.0,0,0,0,0,0,0,0.0,0,1,0,0,1,0,0,1,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,https://www.immowelt.de/expose/2akzn54,Charlottenburg,1340.0,2.0,85.07,Charlottenburg,1000 to 1500,0,0.0,no information,1880.0,no information,0.0,0,0,0,0,0,0,0.0,0,0,1,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0
4,https://www.immowelt.de/expose/29yzr5s,Neukölln,3258.0,4.0,181.95,Neukölln,over 3000,Kelleranteil,0.0,new building,2021.0,no information,0.0,0,0,0,0,0,0,0.0,1,0,1,0,0,0,1,1,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


### cleaning berlin area

In [203]:
df_new["area"].unique()

array(['Kreuzberg', 'Friedrichshain', 'Charlottenburg', 'Neukölln',
       'Tiergarten', 'Köpenick', 'Mitte', 'Schöneberg', 'Lichtenrade',
       'Frohnau', 'Friedrichsfelde', 'Karow', 'Wilhelmsruh',
       'Prenzlauer Berg', 'Heiligensee', 'Waidmannslust', 'Spandau',
       'Französisch Buchholz', 'Wilmersdorf', 'Lichtenberg',
       'Oberschöneweide', 'Altglienicke', 'Moabit', 'Lichterfelde',
       'Friedrichshagen', 'Weißensee', 'Dahlem', 'Zehlendorf',
       'Johannisthal', 'Westend', 'Pankow', 'Steglitz', 'Heinersdorf',
       'Wedding', 'Alt-Hohenschönhausen', 'Hellersdorf',
       'Niederschönhausen', 'Baumschulenweg', 'Karlshorst', 'Tegel',
       'Wilhelmshagen', 'Lankwitz', 'Grunewald', 'Gesundbrunnen',
       'Biesdorf', 'Marzahn', 'Niederschöneweide', 'Halensee',
       'Reinickendorf', 'Hermsdorf', 'Buckow', 'Buch', 'Mariendorf',
       'Adlershof', 'Gropiusstadt', 'Schmargendorf', 'Mahlsdorf',
       'Rahnsdorf', 'Kaulsdorf', 'Marienfelde', 'Wittenau', 'Friedenau',
     

In [207]:
def berlin_area(df):
    mitte = ["Mitte","Moabit","Hansaviertel","Tiergarten",
            "Wedding","Gesundbrunnen"]
    fr_kberg = ["Friedrichshain","Kreuzberg"]
    pankow = ["Prenzlauer Berg","Weißensee","Blankenburg",
            "Heinersdorf","Karow","Stadtrandsiedlung Malchow",
            "Pankow","Blankenfelde","Buch","Französisch Buchholz",
            "Niederschönhausen","Rosenthal","Wilhelmsruh"]
    charl_w = ["Charlottenburg","Wilmersdorf","Schmargendorf",
            "Grunewald","Westend","Charlottenburg-Nord","Halensee"]
    spandau = ["Spandau","Haselhorst","Siemensstadt","Staaken",
            "Gatow","Kladow","Hakenfelde","Falkenhagener Feld",
            "Wilhelmstadt"]
    stegl_z = ["Steglitz","Lichterfelde","Lankwitz","Zehlendorf",
            "Dahlem","Nikolassee","Wannsee","Schlachtensee"]
    temp_sch = ["Schöneberg","Friedenau","Tempelhof","Mariendorf",
            "Marienfelde","Lichtenrade"]
    neukln = ["Neukölln","Britz","Buckow","Rudow","Gropiusstadt"]
    trept_k = ["Alt-Treptow","Plänterwald","Baumschulenweg",
            "Johannisthal","Niederschöneweide","Altglienicke",
            "Adlershof","Bohnsdorf","Oberschöneweide","Köpenick",
            "Friedrichshagen","Rahnsdorf","Grünau","Müggelheim",
            "Schmöckwitz","Karolinenhof", "Wilhelmshagen"]
    mar_hel = ["Marzahn","Biesdorf","Kaulsdorf","Mahlsdorf",
            "Hellersdorf"]
    licht = ["Friedrichsfelde","Karlshorst","Lichtenberg","Falkenberg",
            "Malchow","Wartenberg","Neu-Hohenschönhausen",
            "Alt-Hohenschönhausen","Fennpfuhl","Rummelsburg"]
    rein = ["Reinickendorf","Tegel","Konradshöhe","Heiligensee",
            "Frohnau","Hermsdorf","Waidmannslust","Lübars",
            "Wittenau","Märkisches Viertel","Borsigwalde"]
    area = []
    for i in df["area"]:
        if i in mitte:
            area.append("Mitte")
        elif i in fr_kberg:
            area.append("Friedrichshain-Kreuzberg")
        elif i in pankow:
            area.append("Pankow")
        elif i in charl_w:
            area.append("Charlottenburg-Wilmersdorf")
        elif i in spandau:
            area.append("Spandau")
        elif i in stegl_z:
            area.append("Steglitz-Zehlendorf")
        elif i in temp_sch:
            area.append("Tempelhof-Schöneberg")
        elif i in neukln:
            area.append("Neukölln")
        elif i in trept_k:
            area.append("Treptow-Köpenick")
        elif i in mar_hel:
            area.append("Marzahn-Hellersdorf")
        elif i in licht:
            area.append("Lichtenberg")
        elif i in rein:
            area.append("Reinickendorf")
        else:
            area.append(i)
    return area

In [208]:
area = berlin_area(df_new)
df_new["area"] = area

In [209]:
df_new.area.unique()

array(['Friedrichshain-Kreuzberg', 'Charlottenburg-Wilmersdorf',
       'Neukölln', 'Mitte', 'Treptow-Köpenick', 'Tempelhof-Schöneberg',
       'Reinickendorf', 'Lichtenberg', 'Pankow', 'Spandau',
       'Steglitz-Zehlendorf', 'Marzahn-Hellersdorf', nan], dtype=object)

In [211]:
df_new[df_new.area.isnull()]

,link,addresse,rent,rooms,size,area,rent_range,other_rooms,pets_allowed,condition,year_constructed,furnishing,fully_air_conditioned,window,security,connection,view,services,wellness,partly_air_conditioned,guest_wc,shower,bathtub,bathroom_window,barrier_free,wheelchair_friendly,apartment_sharing,balcony,garden,garden_sharing,terrace,fitted_kitchen,open_kitchen,pantry,parkspace,more_parkspaces,person_elevator,service_elevator,parquet,carpet,tiles,laminate,screed,wooden_floor,synthetic,linoleum,marble
661,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kelleranteil,0.0,new building,2022.0,no information,0.0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
662,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,new building,2017.0,like new,0.0,Kunststofffenster,0,"DSL-Anschluss, Satellitenanschluss",0,0,0,0.0,1,0,1,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
663,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,no information,2021.0,furnished,0.0,0,0,0,0,0,0,0.0,1,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
664,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,no information,2018.0,no information,0.0,0,0,0,0,0,0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
665,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,new building,2023.0,no information,0.0,Holzfenster,0,0,Fernblick,0,0,0.0,1,1,1,1,0,0,0,1,1,0,1,1,1,1,0,0,1,0,0,0,0,0,0,1,0,0,1
666,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Abstellraum, Wasch-Trockenraum",0.0,no information,2016.0,no information,0.0,0,0,0,0,0,0,0.0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [212]:
display(df_new.shape)
df_new = df_new.dropna(axis = 0, subset = "link")
display(df_new.shape)

(660, 47)

(654, 47)

In [213]:
df_new[df_new.area.isnull()]

,link,addresse,rent,rooms,size,area,rent_range,other_rooms,pets_allowed,condition,year_constructed,furnishing,fully_air_conditioned,window,security,connection,view,services,wellness,partly_air_conditioned,guest_wc,shower,bathtub,bathroom_window,barrier_free,wheelchair_friendly,apartment_sharing,balcony,garden,garden_sharing,terrace,fitted_kitchen,open_kitchen,pantry,parkspace,more_parkspaces,person_elevator,service_elevator,parquet,carpet,tiles,laminate,screed,wooden_floor,synthetic,linoleum,marble


In [214]:
df_new.isna().sum()

link                        0
addresse                    0
rent                        0
rooms                       0
size                        0
area                        0
rent_range                  0
other_rooms                 0
pets_allowed                0
condition                   0
year_constructed          121
furnishing                  0
fully_air_conditioned       0
window                      0
security                    0
connection                  0
view                        0
services                    0
wellness                    0
partly_air_conditioned      0
guest_wc                    0
shower                      0
bathtub                     0
bathroom_window             0
barrier_free                0
wheelchair_friendly         0
apartment_sharing           0
balcony                     0
garden                      0
garden_sharing              0
terrace                     0
fitted_kitchen              0
open_kitchen                0
pantry    

### cleaning year_constructed

In [215]:
def year(row):
    if (row["condition"]=="old building") and (row["year_constructed"]=="NaN"):
        return "1900"
    elif (row["condition"]=="new building") and (row["year_constructed"]=="NaN"):
        return "2020"
    elif (row["year_constructed"]==1023):
        return "2023"
    else:
        return row["year_constructed"]

In [216]:
df_new["year_constructed"] = df_new["year_constructed"].fillna("NaN")

In [217]:
import numpy as np
df_new["year"] = df_new.apply(year, axis = 1)


In [218]:
mean_year = round(df_new["year"][df_new["year"]!="NaN"].astype("int64").mean(),0)
mean_year

1983.0

In [219]:
year = []
for i in df_new["year"]:
    if i == "NaN":
        year.append(mean_year)
    else:
        year.append(i)
df_new["year"] = year


In [220]:
df_new["year"].value_counts()

1983.0    115
2022.0     78
2023.0     70
1900.0     61
2021.0     47
         ... 
1962.0      1
1953.0      1
1911.0      1
1899.0      1
1967.0      1
Name: year, Length: 91, dtype: int64

In [221]:
df_new = df_new.drop(columns = ["year_constructed"])
df_new["year"] = df_new["year"].astype("int64")
df_new.head()

,link,addresse,rent,rooms,size,area,rent_range,other_rooms,pets_allowed,condition,furnishing,fully_air_conditioned,window,security,connection,view,services,wellness,partly_air_conditioned,guest_wc,shower,bathtub,bathroom_window,barrier_free,wheelchair_friendly,apartment_sharing,balcony,garden,garden_sharing,terrace,fitted_kitchen,open_kitchen,pantry,parkspace,more_parkspaces,person_elevator,service_elevator,parquet,carpet,tiles,laminate,screed,wooden_floor,synthetic,linoleum,marble,year
0,https://www.immowelt.de/expose/2aqr554,Kreuzberg,620.0,3.0,83.00,Friedrichshain-Kreuzberg,600 to 1000,Kelleranteil,1.0,no information,no information,0.0,0,0,0,0,0,0,0.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1983
1,https://www.immowelt.de/expose/2abw253,Friedrichshain,503.0,2.5,55.00,Friedrichshain-Kreuzberg,below 600,Kelleranteil,1.0,no information,no information,0.0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1983
2,https://www.immowelt.de/expose/2a79752,Stralauer Allee 14a,1750.0,2.0,59.00,Friedrichshain-Kreuzberg,1500 to 2000,Kelleranteil,0.0,renovated,no information,0.0,0,0,0,0,0,0,0.0,0,1,0,0,1,0,0,1,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,2019
3,https://www.immowelt.de/expose/2akzn54,Charlottenburg,1340.0,2.0,85.07,Charlottenburg-Wilmersdorf,1000 to 1500,0,0.0,no information,no information,0.0,0,0,0,0,0,0,0.0,0,0,1,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1880
4,https://www.immowelt.de/expose/29yzr5s,Neukölln,3258.0,4.0,181.95,Neukölln,over 3000,Kelleranteil,0.0,new building,no information,0.0,0,0,0,0,0,0,0.0,1,0,1,0,0,0,1,1,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2021


In [223]:
cols = ["partly_air_conditioned","service_elevator","fully_air_conditioned"]
for i in cols:
    df_new[i] = df_new[i].astype("int64")

In [224]:
rent = []
for i in df_new["rent"]:
    if i < 100:
        i = i*10
        rent.append(i)
    else:
        rent.append(i)
df_new["rent"] = rent

In [225]:
df_new["rent"].min()

248.71

In [226]:
df_new.isna().sum()

link                      0
addresse                  0
rent                      0
rooms                     0
size                      0
area                      0
rent_range                0
other_rooms               0
pets_allowed              0
condition                 0
furnishing                0
fully_air_conditioned     0
window                    0
security                  0
connection                0
view                      0
services                  0
wellness                  0
partly_air_conditioned    0
guest_wc                  0
shower                    0
bathtub                   0
bathroom_window           0
barrier_free              0
wheelchair_friendly       0
apartment_sharing         0
balcony                   0
garden                    0
garden_sharing            0
terrace                   0
fitted_kitchen            0
open_kitchen              0
pantry                    0
parkspace                 0
more_parkspaces           0
person_elevator     

In [227]:
df_new["year"].min()

214

In [233]:
year = []
for i in df_new["year"]:
    if len(str(i)) < 4:
        ii = str(i)[0]+"0"+str(i)[1:]
        ii = int(ii)
        year.append(ii)
    else:
        year.append(i)
min(year)
df_new["year"] = year

In [234]:
df_new.head()

,link,addresse,rent,rooms,size,area,rent_range,other_rooms,pets_allowed,condition,furnishing,fully_air_conditioned,window,security,connection,view,services,wellness,partly_air_conditioned,guest_wc,shower,bathtub,bathroom_window,barrier_free,wheelchair_friendly,apartment_sharing,balcony,garden,garden_sharing,terrace,fitted_kitchen,open_kitchen,pantry,parkspace,more_parkspaces,person_elevator,service_elevator,parquet,carpet,tiles,laminate,screed,wooden_floor,synthetic,linoleum,marble,year
0,https://www.immowelt.de/expose/2aqr554,Kreuzberg,620.0,3.0,83.00,Friedrichshain-Kreuzberg,600 to 1000,Kelleranteil,1.0,no information,no information,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1983
1,https://www.immowelt.de/expose/2abw253,Friedrichshain,503.0,2.5,55.00,Friedrichshain-Kreuzberg,below 600,Kelleranteil,1.0,no information,no information,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1983
2,https://www.immowelt.de/expose/2a79752,Stralauer Allee 14a,1750.0,2.0,59.00,Friedrichshain-Kreuzberg,1500 to 2000,Kelleranteil,0.0,renovated,no information,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,2019
3,https://www.immowelt.de/expose/2akzn54,Charlottenburg,1340.0,2.0,85.07,Charlottenburg-Wilmersdorf,1000 to 1500,0,0.0,no information,no information,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1880
4,https://www.immowelt.de/expose/29yzr5s,Neukölln,3258.0,4.0,181.95,Neukölln,over 3000,Kelleranteil,0.0,new building,no information,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,1,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2021


In [235]:
df_new.isna().sum().sum()

0

In [236]:
df_new.to_csv("data full clean new.csv", index = False)